In [1]:
import pathlib as pl

import kagglehub
import pandas as pd

(mvls_links_path, mvls_genometags_path, mvls_movies_path, mvls_genomescores_path, mvls_tags_path, mvls_ratings_path) = (
  list(pl.Path(kagglehub.dataset_download("grouplens/movielens-20m-dataset")).iterdir())
)
print(
  mvls_links_path, mvls_genometags_path, mvls_movies_path, mvls_genomescores_path, mvls_tags_path, mvls_ratings_path
)

/home/dodo/code/recsys-movies/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/dodo/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/link.csv /home/dodo/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_tags.csv /home/dodo/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/movie.csv /home/dodo/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_scores.csv /home/dodo/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/tag.csv /home/dodo/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/rating.csv


In [2]:
movies = pd.read_csv(mvls_movies_path).set_index("movieId").sort_index()

movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
# build dummy columns for each genre
movies = movies.join(movies["genres"].str.get_dummies(sep="|"))
movies.sample(5, random_state=42)

,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
61116,Black Caesar (1973),Crime|Drama,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
70697,G-Force (2009),Action|Adventure|Children|Fantasy,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111931,Raze (2013),Action|Horror,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
26630,Moonwalker (1988),Musical,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
63692,Don Q Son of Zorro (1925),Adventure|Romance,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
ratings = pd.read_csv(mvls_ratings_path)

# get all ratings for user 0
user_ratings = ratings[ratings["userId"] == 1]
# join with movie information
user_ratings = user_ratings.merge(movies, on="movieId")
user_ratings.head()

,userId,movieId,rating,timestamp,title,genres,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [5]:
# get a single list of all unique genres for all movies
genres = list(set(movies["genres"].str.split("|").sum()))
genres = sorted(genres)
genres

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [6]:
# multiply the genre columns by the rating
user_ratings[genres] = user_ratings[genres].multiply(user_ratings["rating"], axis="index")
user_ratings

,userId,movieId,rating,timestamp,title,genres,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy,0.0,0.0,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,0.0,0.0,3.5,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,3.5,0.0,0.0,0.0
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,3.5,3.5,0.0,0.0
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,1,8507,5.0,2004-09-10 03:13:47,Freaks (1932),Crime|Drama|Horror,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,1,8636,4.5,2005-04-02 23:44:53,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,0.0,4.5,4.5,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,4.5,0.0,0.0,0.0
172,1,8690,3.5,2005-04-02 23:33:15,Slaughterhouse-Five (1972),Comedy|Drama|Sci-Fi|War,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,3.5,0.0
173,1,8961,4.0,2005-04-02 23:47:09,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,0.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
user_genre_preferences = user_ratings[genres].sum() / user_ratings[genres].sum().sum()
user_genre_preferences

(no genres listed)    0.000000
Action                0.127230
Adventure             0.143005
Animation             0.018878
Children              0.035428
Comedy                0.079131
Crime                 0.041376
Documentary           0.000000
Drama                 0.083786
Fantasy               0.135247
Film-Noir             0.000000
Horror                0.087148
IMAX                  0.004396
Musical               0.005689
Mystery               0.033618
Romance               0.022498
Sci-Fi                0.076804
Thriller              0.081717
War                   0.017067
Western               0.006982
dtype: float64

In [8]:
# get the movies that the user has not seen
unseen_movies = movies[~movies.index.isin(user_ratings["movieId"])]
unseen_movies

,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Heat (1995),Action|Crime|Thriller,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131254,Kein Bund für's Leben (2007),Comedy,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
131256,"Feuer, Eis & Dosenbier (2002)",Comedy,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
131258,The Pirates (2014),Adventure,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# multiply the user genre preferences by the unseen movies
unseen_movies[genres].multiply(user_genre_preferences)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0.0,0.00000,0.143005,0.018878,0.035428,0.079131,0.000000,0.0,0.000000,0.135247,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.00000,0.000000,0.000000,0.000000,0.079131,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.022498,0.0,0.000000,0.0,0.0
4,0.0,0.00000,0.000000,0.000000,0.000000,0.079131,0.000000,0.0,0.083786,0.000000,0.0,0.000000,0.0,0.0,0.0,0.022498,0.0,0.000000,0.0,0.0
5,0.0,0.00000,0.000000,0.000000,0.000000,0.079131,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
6,0.0,0.12723,0.000000,0.000000,0.000000,0.000000,0.041376,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.081717,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131254,0.0,0.00000,0.000000,0.000000,0.000000,0.079131,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
131256,0.0,0.00000,0.000000,0.000000,0.000000,0.079131,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
131258,0.0,0.00000,0.143005,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [10]:
unseen_movies["content_based_rating"] = unseen_movies[genres].multiply(user_genre_preferences).sum(axis=1)

/tmp/ipykernel_3183/461131408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unseen_movies["content_based_rating"] = unseen_movies[genres].multiply(user_genre_preferences).sum(axis=1)


In [11]:
unseen_movies[["title", "content_based_rating"]].sort_values("content_based_rating", ascending=False)

,title,content_based_rating
movieId,,
81132,Rubber (2010),0.683993
49593,She (1965),0.675718
5018,Motorama (1991),0.674683
71999,Aelita: The Queen of Mars (Aelita) (1924),0.670287
72165,Cirque du Freak: The Vampire's Assistant (2009),0.653478
...,...,...
100246,Pretty Sweet (2012),0.000000
99,Heidi Fleiss: Hollywood Madam (1995),0.000000
100266,Day Is Done (2011),0.000000


In [12]:
(unseen_movies["content_based_rating"] != 0).sum() / len(unseen_movies)

0.9187543814337896